# Import modules

In [243]:
import numpy as np
import pingouin as pg
import pandas as pd
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statsmodels.api as sm
import statsmodels.formula.api as smf


# Functions (helpers)

In [244]:
def read_csv(name, info=True):
    df = pd.read_csv(f'../data/stats/{name}.csv')
    # pd.set_option('display.max_columns', None)
    # pd.set_option('max_columns', None)
    if info:
        print_section('DATASET INFO')
        print('N rows=', df.size)
        print('N cols=', len(df.columns))
        print(df.head())
    return df

def print_section(title):
    print( '*' + '-' * 150 + '*')
    print(title)
    print( '*' + '-' * 150 + '*')

# Fig 2

 * load data

In [245]:
filename = 'Fig2A'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format


*------------------------------------------------------------------------------------------------------------------------------------------------------*
DATASET INFO
*------------------------------------------------------------------------------------------------------------------------------------------------------*
N rows= 7672
N cols= 7
   subject  exp_num  complete  block  less_cues   score  cond
0        1        1         0      0          0 0.16667     4
1        2        1         0      0          0 0.86667     4
2        3        1         0      0          0 0.46667     4
3        4        1         0      0          0 0.40000     4
4        5        1         0      0          0 0.73333     4


* Test for performance above chance in experiments 1:4 (Fig. 2A)

In [246]:
table = []
for exp_num in (1, 2, 3, 4):
    for cond in (1, 2, 3, 4):
        if exp_num == 4 and cond not in (1, 4):
            continue
        row = pg.ttest(df['score'][df.exp_num==exp_num][df.cond==cond], 0.5)
        row.insert(loc=0, column='exp_num', value=exp_num)
        row.insert(loc=1, column='cond', value=cond)
        table.append(row)

df2 = pd.concat(table)
df2

,exp_num,cond,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1,1,10.98380,75,two-sided,0.00000,"[0.71, 0.8]",1.25993,2.102e+14,1.00000
T-test,1,2,6.99011,75,two-sided,0.00000,"[0.63, 0.73]",0.80182,1.093e+07,1.00000
T-test,1,3,7.00086,75,two-sided,0.00000,"[0.62, 0.72]",0.80305,1.143e+07,1.00000
T-test,1,4,1.50661,75,two-sided,0.13611,"[0.49, 0.6]",0.17282,0.372,0.31852
T-test,2,1,19.43878,70,two-sided,0.00000,"[0.8, 0.87]",2.30696,4.82e+26,1.00000
T-test,2,2,11.65526,70,two-sided,0.00000,"[0.71, 0.8]",1.38323,1.206e+15,1.00000
T-test,2,3,7.72376,70,two-sided,0.00000,"[0.63, 0.72]",0.91664,1.62e+08,1.00000
T-test,2,4,3.62985,70,two-sided,0.00054,"[0.54, 0.64]",0.43078,44.471,0.94735
T-test,3,1,27.05025,82,two-sided,0.00000,"[0.88, 0.94]",2.96915,1.549e+39,1.00000
T-test,3,2,19.91188,82,two-sided,0.00000,"[0.82, 0.9]",2.18561,7.365e+29,1.00000


* Difficult vs Easy choice problems pairwise ttests

In [247]:
df
pg.pairwise_tests(data=df, dv='score',  subject='subject', between='exp_num', within='cond', padjust='bonf', effsize='cohen', within_first=False)

,subject,exp_num,complete,block,less_cues,score,cond
0,1,1,0,0,0,0.16667,4
1,2,1,0,0,0,0.86667,4
2,3,1,0,0,0,0.46667,4
3,4,1,0,0,0,0.40000,4
4,5,1,0,0,0,0.73333,4
...,...,...,...,...,...,...,...
1091,314,4,1,1,1,0.76667,1
1092,315,4,1,1,1,0.98333,1
1093,316,4,1,1,1,0.81667,1
1094,317,4,1,1,1,0.86667,1


,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,1.00000,2.00000,False,True,-2.69270,143.24161,two-sided,0.00793,0.02380,bonf,4.704,-0.44173
1,exp_num,-,1.00000,3.00000,False,True,-7.65359,156.48001,two-sided,0.00000,0.00000,bonf,3.523e+09,-1.21345
2,exp_num,-,2.00000,3.00000,False,True,-5.48549,151.56326,two-sided,0.00000,0.00000,bonf,7.315e+04,-0.87240
3,cond,-,1.00000,2.00000,True,True,5.74184,229.00000,two-sided,0.00000,0.00000,bonf,3.014e+05,0.35073
4,cond,-,1.00000,3.00000,True,True,9.04336,229.00000,two-sided,0.00000,0.00000,bonf,7.78e+13,0.61669
5,cond,-,1.00000,4.00000,True,True,11.60547,229.00000,two-sided,0.00000,0.00000,bonf,4.361e+21,0.96845
6,cond,-,2.00000,3.00000,True,True,3.40183,229.00000,two-sided,0.00079,0.00474,bonf,19.493,0.25176
7,cond,-,2.00000,4.00000,True,True,7.42745,229.00000,two-sided,0.00000,0.00000,bonf,3.032e+09,0.61596
8,cond,-,3.00000,4.00000,True,True,4.42752,229.00000,two-sided,0.00001,0.00009,bonf,801.388,0.38100
9,exp_num * cond,1,1,2,True,True,2.82718,75.00000,two-sided,0.00602,0.10832,bonf,4.973,0.36470


* Test for performance within Exp. 1:4 across conditions (pairs of symbols) (Fig. 2A)

In [248]:
# test performance within exp and across cond with regression
print_section('regress performance across conditions within experiments')

x = df[['cond', 'exp_num']]
y = df[['exp_num', 'score']]

table = []
for exp_num in (1, 2, 3, 4):
    
    row = pg.linear_regression(
        x[x.exp_num==exp_num].cond, y[y.exp_num==exp_num].score, add_intercept=True, as_dataframe=False)

    row['dof_model'] = [row['df_model'], ] * len(row['coef'])
    row['dof_resid'] = [row['df_resid'], ] * len(row['coef'])

    del row['residuals']; del row['df_model'];
    del row['df_resid']; del row['X']; del row['y']; del row['pred']
    row = pd.DataFrame(row)
    
    row.insert(loc=0, column='exp_num', value=exp_num)
    table.append(
        row
    )

df3 = pd.concat(table)
df3

# ANOVA control
print_section('Anova performance across conditions within experiments')
table = []
for exp_num in (1, 2, 3, 4):
    
    row = pg.rm_anova(data=df, dv='score', within='cond', detailed=True, subject='subject', correction=True)

    row = pd.DataFrame(row)
    
    row.insert(loc=0, column='exp_num', value=exp_num)
    table.append(
        row
    )

df4 = pd.concat(table)
df4




*------------------------------------------------------------------------------------------------------------------------------------------------------*
regress performance across conditions within experiments
*------------------------------------------------------------------------------------------------------------------------------------------------------*


,exp_num,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%],dof_model,dof_resid
0,1,Intercept,0.82193,0.03102,26.49265,0.00000,0.09662,0.09363,0.76088,0.88298,1,302
1,1,cond,-0.06439,0.01133,-5.68346,0.00000,0.09662,0.09363,-0.08668,-0.04209,1,302
0,2,Intercept,0.92347,0.02662,34.69104,0.00000,0.20638,0.20357,0.87108,0.97587,1,282
1,2,cond,-0.08324,0.00972,-8.56352,0.00000,0.20638,0.20357,-0.10237,-0.06411,1,282
0,3,Intercept,0.96486,0.02540,37.98767,0.00000,0.09879,0.09606,0.91489,1.01482,1,330
1,3,cond,-0.05578,0.00927,-6.01467,0.00000,0.09879,0.09606,-0.07403,-0.03754,1,330
0,4,Intercept,0.96521,0.03172,30.42501,0.00000,0.18856,0.18390,0.90260,1.02783,1,174
1,4,cond,-0.06919,0.01088,-6.35875,0.00000,0.18856,0.18390,-0.09067,-0.04772,1,174


*------------------------------------------------------------------------------------------------------------------------------------------------------*
Anova performance across conditions within experiments
*------------------------------------------------------------------------------------------------------------------------------------------------------*


,exp_num,Source,SS,DF,MS,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,1,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,1,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,2,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,3,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,cond,5.22892,3,1.74297,56.85023,0.00000,0.00000,0.11422,0.80801,False,0.67885,0.00000
1,4,Error,21.06275,687,0.03066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Test performance increase across Exp 1:4 (Fig. 2A)

In [249]:
# test performance within exp and across cond with regression
print_section('regress performance across experiments')
df_ind = df.groupby('subject').mean()
# df_ind
smf.ols('score ~ C(exp_num)', data=df_ind).fit().summary()
# res.summary()

*------------------------------------------------------------------------------------------------------------------------------------------------------*
regress performance across experiments
*------------------------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     21.02
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           1.96e-12
Time:                        16:03:13   Log-Likelihood:                 166.18
No. Observations:                 318   AIC:                            -324.4
Df Residuals:                     314   BIC:                            -309.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.6610      0.017     39.905      0.000       0.628       0.694
C(exp_num)[T.2.0]     0.0544      0.024      2.283      0.023       0.008       0.101
C(exp_num)[T.3.0]     0.1644      0.023      7.173      0.000       0.119       0.210
C(exp_num)[T.4.0]     0.1313      0.023      5.805      0.000       0.087       0.176
==============================================================================
Omnibus:                       55.410   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              105.354
Skew:                          -0.936   Prob(JB):                     1.33e-23
Kurtosis:                       5.108   Cond. No.                         4.90
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* Mean accuracy for exp 1-4

In [250]:
# print mean and variance of each exp
table = [] 
for exp_num in (1, 2, 3, 4):
    table.append(
        {
            'Exp. ': exp_num,
            'mean': df_ind[df_ind.exp_num==exp_num].score.mean(),
            'sem': df_ind[df_ind.exp_num==exp_num].score.sem(),
        }
    )
pd.DataFrame(table)
            

,Exp.,mean,sem
0,1,0.66096,0.01529
1,2,0.71538,0.01321
2,3,0.82540,0.01509
3,4,0.79223,0.01911



* Fig 2D, LE-ES-SP slope increase

In [251]:
filename = 'Fig2D'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format
# test performance within exp and across cond with regression
print_section('regress slope across experiments')

# iterate through each modality and set it as "reference modality" to get "absolute slopes" for each
for modality in ('LE', 'ES', 'SP'):
    print_section('Modality used as reference= ' + modality)
    smf.ols(f'slope ~ C(exp_num) * C(modality, Treatment("{modality}"))', data=df).fit().summary()



*------------------------------------------------------------------------------------------------------------------------------------------------------*
DATASET INFO
*------------------------------------------------------------------------------------------------------------------------------------------------------*
N rows= 3816
N cols= 4
   subject  exp_num   slope modality
0        1        1 0.42728       LE
1        2        1 0.90420       LE
2        3        1 0.43349       LE
3        4        1 0.65978       LE
4        5        1 0.84207       LE
*------------------------------------------------------------------------------------------------------------------------------------------------------*
regress slope across experiments
*------------------------------------------------------------------------------------------------------------------------------------------------------*
*------------------------------------------------------------------------------------------------

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  slope   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     43.28
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           3.31e-76
Time:                        16:03:13   Log-Likelihood:                -359.93
No. Observations:                 954   AIC:                             743.9
Df Residuals:                     942   BIC:                             802.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              0.5900      0.041     14.484      0.000       0.510       0.670
C(exp_num)[T.2]                                        0.1119      0.059      1.909      0.057      -0.003       0.227
C(exp_num)[T.3]                                        0.2854      0.056      5.062      0.000       0.175       0.396
C(exp_num)[T.4]                                        0.3145      0.056      5.656      0.000       0.205       0.424
C(modality, Treatment("LE"))[T.ES]                    -0.3650      0.058     -6.337      0.000      -0.478      -0.252
C(modality, Treatment("LE"))[T.SP]                    -0.2704      0.058     -4.694      0.000      -0.383      -0.157
C(exp_num)[T.2]:C(modality, Treatment("LE"))[T.ES]    -0.2131      0.083     -2.571      0.010      -0.376      -0.050
C(exp_num)[T.3]:C(modality, Treatment("LE"))[T.ES]    -0.2617      0.080     -3.283      0.001      -0.418      -0.105
C(exp_num)[T.4]:C(modality, Treatment("LE"))[T.ES]    -0.1977      0.079     -2.514      0.012      -0.352      -0.043
C(exp_num)[T.2]:C(modality, Treatment("LE"))[T.SP]    -0.2057      0.083     -2.482      0.013      -0.368      -0.043
C(exp_num)[T.3]:C(modality, Treatment("LE"))[T.SP]    -0.1936      0.080     -2.428      0.015      -0.350      -0.037
C(exp_num)[T.4]:C(modality, Treatment("LE"))[T.SP]    -0.1332      0.079     -1.693      0.091      -0.287       0.021
==============================================================================
Omnibus:                       28.676   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.750
Skew:                           0.387   Prob(JB):                     1.28e-07
Kurtosis:                       3.448   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

*------------------------------------------------------------------------------------------------------------------------------------------------------*
Modality used as reference= ES
*------------------------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  slope   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     43.28
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           3.31e-76
Time:                        16:03:14   Log-Likelihood:                -359.93
No. Observations:                 954   AIC:                             743.9
Df Residuals:                     942   BIC:                             802.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              0.2250      0.041      5.523      0.000       0.145       0.305
C(exp_num)[T.2]                                       -0.1012      0.059     -1.727      0.084      -0.216       0.014
C(exp_num)[T.3]                                        0.0237      0.056      0.420      0.675      -0.087       0.134
C(exp_num)[T.4]                                        0.1169      0.056      2.102      0.036       0.008       0.226
C(modality, Treatment("ES"))[T.LE]                     0.3650      0.058      6.337      0.000       0.252       0.478
C(modality, Treatment("ES"))[T.SP]                     0.0946      0.058      1.643      0.101      -0.018       0.208
C(exp_num)[T.2]:C(modality, Treatment("ES"))[T.LE]     0.2131      0.083      2.571      0.010       0.050       0.376
C(exp_num)[T.3]:C(modality, Treatment("ES"))[T.LE]     0.2617      0.080      3.283      0.001       0.105       0.418
C(exp_num)[T.4]:C(modality, Treatment("ES"))[T.LE]     0.1977      0.079      2.514      0.012       0.043       0.352
C(exp_num)[T.2]:C(modality, Treatment("ES"))[T.SP]     0.0074      0.083      0.089      0.929      -0.155       0.170
C(exp_num)[T.3]:C(modality, Treatment("ES"))[T.SP]     0.0682      0.080      0.855      0.393      -0.088       0.225
C(exp_num)[T.4]:C(modality, Treatment("ES"))[T.SP]     0.0645      0.079      0.820      0.412      -0.090       0.219
==============================================================================
Omnibus:                       28.676   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.750
Skew:                           0.387   Prob(JB):                     1.28e-07
Kurtosis:                       3.448   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

*------------------------------------------------------------------------------------------------------------------------------------------------------*
Modality used as reference= SP
*------------------------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  slope   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     43.28
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           3.31e-76
Time:                        16:03:14   Log-Likelihood:                -359.93
No. Observations:                 954   AIC:                             743.9
Df Residuals:                     942   BIC:                             802.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              0.3196      0.041      7.847      0.000       0.240       0.400
C(exp_num)[T.2]                                       -0.0938      0.059     -1.600      0.110      -0.209       0.021
C(exp_num)[T.3]                                        0.0918      0.056      1.629      0.104      -0.019       0.202
C(exp_num)[T.4]                                        0.1814      0.056      3.262      0.001       0.072       0.291
C(modality, Treatment("SP"))[T.ES]                    -0.0946      0.058     -1.643      0.101      -0.208       0.018
C(modality, Treatment("SP"))[T.LE]                     0.2704      0.058      4.694      0.000       0.157       0.383
C(exp_num)[T.2]:C(modality, Treatment("SP"))[T.ES]    -0.0074      0.083     -0.089      0.929      -0.170       0.155
C(exp_num)[T.3]:C(modality, Treatment("SP"))[T.ES]    -0.0682      0.080     -0.855      0.393      -0.225       0.088
C(exp_num)[T.4]:C(modality, Treatment("SP"))[T.ES]    -0.0645      0.079     -0.820      0.412      -0.219       0.090
C(exp_num)[T.2]:C(modality, Treatment("SP"))[T.LE]     0.2057      0.083      2.482      0.013       0.043       0.368
C(exp_num)[T.3]:C(modality, Treatment("SP"))[T.LE]     0.1936      0.080      2.428      0.015       0.037       0.350
C(exp_num)[T.4]:C(modality, Treatment("SP"))[T.LE]     0.1332      0.079      1.693      0.091      -0.021       0.287
==============================================================================
Omnibus:                       28.676   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.750
Skew:                           0.387   Prob(JB):                     1.28e-07
Kurtosis:                       3.448   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* Fig 2D, LE-ES-SP slope comparisons

In [252]:
filename = 'Fig2D'
df = read_csv(filename)
pd.options.display.float_format = '{:,.5f}'.format
pg.pairwise_tests(dv='slope', subject='subject', between='exp_num', within='modality', effsize='cohen', data=df, padjust='bonf', parametric=True, within_first=False)

*------------------------------------------------------------------------------------------------------------------------------------------------------*
DATASET INFO
*------------------------------------------------------------------------------------------------------------------------------------------------------*
N rows= 3816
N cols= 4
   subject  exp_num   slope modality
0        1        1 0.42728       LE
1        2        1 0.90420       LE
2        3        1 0.43349       LE
3        4        1 0.65978       LE
4        5        1 0.84207       LE


,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,1,2,False,True,0.71682,134.60910,two-sided,0.47473,1.00000,bonf,0.225,0.11693
1,exp_num,-,1,3,False,True,-3.14624,154.33538,two-sided,0.00198,0.01191,bonf,15.065,-0.50046
2,exp_num,-,1,4,False,True,-4.59750,161.26770,two-sided,0.00001,0.00005,bonf,1945.049,-0.71571
3,exp_num,-,2,3,False,True,-4.41133,148.51098,two-sided,0.00002,0.00012,bonf,931.038,-0.69631
4,exp_num,-,2,4,False,True,-5.97435,150.28953,two-sided,0.00000,0.00000,bonf,7.021e+05,-0.91179
5,exp_num,-,3,4,False,True,-1.65671,168.31190,two-sided,0.09944,0.59664,bonf,0.588,-0.25258
6,modality,-,ES,LE,True,True,-21.46513,317.00000,two-sided,0.00000,0.00000,bonf,1.54e+60,-1.61196
7,modality,-,ES,SP,True,True,-7.36087,317.00000,two-sided,0.00000,0.00000,bonf,3.713e+09,-0.33217
8,modality,-,LE,SP,True,True,14.85496,317.00000,two-sided,0.00000,0.00000,bonf,8.099e+34,1.08016
9,exp_num * modality,1,ES,LE,True,True,-6.50564,75.00000,two-sided,0.00000,0.00000,bonf,1.503e+06,-1.03599


# Fig 3

* load data
 

In [253]:
filename = 'Fig3D'
df = read_csv(filename)


*------------------------------------------------------------------------------------------------------------------------------------------------------*
DATASET INFO
*------------------------------------------------------------------------------------------------------------------------------------------------------*
N rows= 1624
N cols= 4
   subject  exp_num   slope modality
0        1  5.00000 0.06667       ES
1        2  5.00000 1.33418       ES
2        3  5.00000 0.05210       ES
3        4  5.00000 0.53807       ES
4        5  5.00000 0.92935       ES


* Fig3D, ES-EE slope ttest comparisons

In [254]:
pg.pairwise_tests(data=df, dv='slope', subject='subject', between='exp_num', within='modality', padjust='bonf', within_first=False, effsize='cohen')

,Contrast,exp_num,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,exp_num,-,5.00000,6.10000,False,True,-0.01296,134.93769,two-sided,0.98968,1.00000,bonf,0.183,-0.00221
1,exp_num,-,5.00000,6.20000,False,True,-1.38826,134.69286,two-sided,0.16735,0.50204,bonf,0.44,-0.23632
2,exp_num,-,6.10000,6.20000,False,True,-1.38645,130.00000,two-sided,0.16798,0.50395,bonf,0.446,-0.24135
3,modality,-,EE,ES,True,True,7.68370,202.00000,two-sided,0.00000,NaN,NaN,1.007e+10,0.50407
4,exp_num * modality,5.00000,EE,ES,True,True,4.48225,70.00000,two-sided,0.00003,0.00008,bonf,667.492,0.53396
5,exp_num * modality,6.10000,EE,ES,True,True,4.05588,65.00000,two-sided,0.00014,0.00041,bonf,159.138,0.40360
6,exp_num * modality,6.20000,EE,ES,True,True,4.84340,65.00000,two-sided,0.00001,0.00002,bonf,2136.461,0.57446



* Fig 3D, ES-EE slope regression across Exp 6.1-6.2

In [255]:
# test performance within exp and across cond with regression
print_section('regress slope across experiments')

df['exp_num'] = df['exp_num'].astype(str)
df = df[df.exp_num!='5.0']
# iterate through each modality and set it as "reference modality" to get "absolute slopes" for each
for modality in ('ES', 'EE'):
    print_section('Modality used as reference= ' + modality)
    smf.ols(f'slope ~ C(exp_num, Treatment("6.1")) * C(modality, Treatment("{modality}"))', data=df).fit().summary()

*------------------------------------------------------------------------------------------------------------------------------------------------------*
regress slope across experiments
*------------------------------------------------------------------------------------------------------------------------------------------------------*
*------------------------------------------------------------------------------------------------------------------------------------------------------*
Modality used as reference= ES
*------------------------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  slope   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     6.402
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           0.000336
Time:                        16:03:14   Log-Likelihood:                -101.86
No. Observations:                 264   AIC:                             211.7
Df Residuals:                     260   BIC:                             226.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                  0.3163      0.044      7.165      0.000       0.229       0.403
C(exp_num, Treatment("6.1"))[T.6.2]                                        0.0493      0.062      0.789      0.431      -0.074       0.172
C(modality, Treatment("ES"))[T.EE]                                         0.1466      0.062      2.349      0.020       0.024       0.270
C(exp_num, Treatment("6.1"))[T.6.2]:C(modality, Treatment("ES"))[T.EE]     0.0566      0.088      0.641      0.522      -0.117       0.230
==============================================================================
Omnibus:                       15.461   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.088
Skew:                          -0.530   Prob(JB):                     0.000195
Kurtosis:                       3.656   Cond. No.                         6.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

*------------------------------------------------------------------------------------------------------------------------------------------------------*
Modality used as reference= EE
*------------------------------------------------------------------------------------------------------------------------------------------------------*


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  slope   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     6.402
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           0.000336
Time:                        16:03:14   Log-Likelihood:                -101.86
No. Observations:                 264   AIC:                             211.7
Df Residuals:                     260   BIC:                             226.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                  0.4629      0.044     10.487      0.000       0.376       0.550
C(exp_num, Treatment("6.1"))[T.6.2]                                        0.1059      0.062      1.696      0.091      -0.017       0.229
C(modality, Treatment("EE"))[T.ES]                                        -0.1466      0.062     -2.349      0.020      -0.270      -0.024
C(exp_num, Treatment("6.1"))[T.6.2]:C(modality, Treatment("EE"))[T.ES]    -0.0566      0.088     -0.641      0.522      -0.230       0.117
==============================================================================
Omnibus:                       15.461   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.088
Skew:                          -0.530   Prob(JB):                     0.000195
Kurtosis:                       3.656   Cond. No.                         6.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Review, fig1, test accuracy bad/good

In [256]:
filename = 'corr_tables'
df = read_csv(filename)
pg.pairwise_ttests(dv='slope', subject='subject', between='exp_num', within='modality', 
            data=df, padjust='bonf', within_first=False, parametric=True)

# pg.print_table(res, floatfmt='.6f')




*------------------------------------------------------------------------------------------------------------------------------------------------------*
DATASET INFO
*------------------------------------------------------------------------------------------------------------------------------------------------------*
N rows= 592
N cols= 4
   subject  exp_num    slope modality
0        1        1  0.22351       ES
1        2        1  0.44697       ES
2        3        1 -0.06116       ES
3        4        1  0.52977       ES
4        5        1  0.23695       ES


/home/getz/.local/lib/python3.8/site-packages/pingouin/pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


ValueError: Columns must have at least two unique values.

# Fig 2, test slope bad good

In [ ]:
filename = 'review_perf_fig2'
df = read_csv(filename)

pg.pairwise_ttests(dv='value', between=['exp_num', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)





In [ ]:
filename = 'review_accuracy'
df = read_csv(filename)

res = pg.pairwise_ttests(dv='value', between=['modality', 'split'], subject='id',
            data=df, padjust='bonf', parametric=True)

pg.print_table(res, floatfmt='.6f')


